In [1]:
# initial preprocessing 
# 1. lang dtection 
# 2. lang translation


In [63]:
# importing libraries 
import pandas as pd 
from langdetect import detect
from googletrans import Translator
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud
from nltk.util import ngrams
from rake_nltk import Rake
import yake 
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import silhouette_score
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import contractions
from string import punctuation
from unidecode import unidecode
from nltk.stem import WordNetLemmatizer
import numpy as np

In [64]:
data = pd.read_csv("dataset_es_train.csv")
data.head(2)

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0491108,product_es_0296024,reviewer_es_0999081,1,Nada bueno se me fue ka pantalla en menos de 8...,television Nevir,es,electronics
1,es_0869872,product_es_0922286,reviewer_es_0216771,1,"Horrible, nos tuvimos que comprar otro porque ...",Dinero tirado a la basura con esta compra,es,electronics


In [65]:
# np.random.seed(42)

final_data = data.sample(1000)

In [66]:
final_data = final_data[['review_body','stars']]

In [67]:
final_data.reset_index(drop=True,inplace=True)

In [68]:
final_data

,review_body,stars
0,"Es pequeña para mi MacBook Pro de 13, y me man...",1
1,"Lo devolví no tiene 1,5 cm de espesor el que m...",1
2,Llegaron con mucho retraso. No son de buena ca...,2
3,Es lo que dice y me gusta especialmente que so...,5
4,"es un elemento simple, de buena calidad de aca...",5
...,...,...
995,"Me gusta la funcionalidad y su resistencia, bu...",4
996,Esta bien aunque tiene poca variedad de textur...,2
997,No consigo conectarla con la aplicacion. He pr...,1
998,"Pequeño , la talla es muy pequeña , pedimos la...",1


In [69]:
final_data.dtypes

review_body    object
stars           int64
dtype: object

In [70]:
def lang_detection(data):
    lang = detect(data)
    return lang

In [71]:
final_data['language'] = final_data['review_body'].apply(lang_detection)

In [72]:
final_data['language'].value_counts()

language
es    974
pt     13
en      7
ca      3
so      2
de      1
Name: count, dtype: int64

In [79]:
# # lang translation 
# def lang_translation(data):
#     translator = Translator()
#     clean_text = translator.translate(data)
#     return clean_text.text

In [88]:
from translate import Translator

def lang_translation(text, src_lang='es', dest_lang='en'):
    """
    Translates a given text to English using an offline translation model.

    Parameters:
        text (str): The text to be translated.
        src_lang (str): The source language code (e.g., 'es' for Spanish).
        dest_lang (str): The destination language code (e.g., 'en' for English).

    Returns:
        str: The translated text in English.
    """
    translator = Translator(from_lang=src_lang, to_lang=dest_lang)
    translated_text = translator.translate(text)
    return translated_text

In [89]:
final_data['review_body'][0]

'Es pequeña para mi MacBook Pro de 13, y me mandaron otro dibujo, lo raro es que si busco por el código de barras me devuelve el que yo pedí, por lo que estimo que está mal etiquetada. La pediré de nuevo a ver si la mandan bien. Llegó a tiempo, como siempre.'

In [90]:
lang_translation(final_data['review_body'][0])

"It is small for my MacBook Pro of 13, and they sent me another drawing, the strange thing is that if I look for the barcode it returns the one I ordered, so I think it is mislabeled. I'll ask her again to see if they send her well. He arrived on time, as always."

In [16]:
final_data['review_body'][1]

'El cable es muy largo y es comodo pero me duró poco tiempo porque el conector microusb se me acabó doblando'

In [ ]:
lang_translation(final_data['review_body'][1])

In [ ]:
final_data['translated_review'] = final_data['review_body'].apply(lang_translation)

In [ ]:
final_data.stars

In [ ]:
stopwordlist =  stopwords.words("english")
stopwordlist.remove('not')
stopwordlist.remove('nor')
stopwordlist.remove('no')

In [ ]:
def extract_ngrams(data,ngram_range):
    tokens = [word.lower() for word in word_tokenize(data) if (word not in punctuation) and (word.isalpha()) and (word.lower() not in stopwordlist)]
    ngrams_list = ngrams(tokens,ngram_range)
    final_ngrams = []
    for words in ngrams_list:
        final_ngrams.append(" ".join(words))
    return final_ngrams

unigram_list = final_data.translated_review.apply(lambda x :extract_ngrams(x,1) )

In [ ]:
final_unigram = []
for unigram in unigram_list:
    final_unigram.extend(unigram)
top_25_unigrams = Counter(final_unigram).most_common(100)
top_25_unigrams

In [ ]:
# wordcloud  : ""
def generate_wordcloud(data,column):
    df = data[column].str.cat(sep=" ") # record1   record 2 
    text = " ".join([word for word in df.split()])# 
    word_cloud = WordCloud(height=500,width=700,background_color='white',min_font_size=10).generate(text)
    plt.figure(figsize=(10,16))
    plt.imshow(word_cloud)
    plt.axis("off")
    plt.show()

In [ ]:
generate_wordcloud(final_data,'translated_review')

In [ ]:
# # 1. remove blanklines,whitespaces,tabs 
def remove_spaces(data):
    "Function will remove all spaces"
    formatted_text = data.replace('\\n',' ').replace('\t',' ').replace("\\",' ')
    return formatted_text 
# 2. Contraction mapping 
def expand_text(data):
    fixed_text = contractions.fix(data)
    return fixed_text

# 3. Handling accented 
def handling_accented(data):
    fixed_text = unidecode(data)
    return fixed_text
# 4.cleaning 
stopword_list = stopwords.words("english")
stopword_list.remove("not")
stopword_list.remove("nor")
stopword_list.remove("no")
def cleaning(data):
    tokens = word_tokenize(data) # tokenization
    clean_text = [word.lower() for word in tokens if (word not in stopword_list) and(word not in punctuation) and(len(word)>2) and(word.isalpha())]
    return clean_text

# 5. autocorrect 
def autocorrection(data):
    spell = Speller(lang='en')
    corrected_text = spell(data)
    return corrected_text

# 6. Lemmatization 
def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    final_text = []
    for word in data:
        lemmatized_word = lemmatizer.lemmatize(word)
        final_text.append(lemmatized_word)
    return " ".join(final_text)

In [ ]:
clean_text = final_data.translated_review.apply(remove_spaces)
clean_text = clean_text.apply(expand_text)
clean_text = clean_text.apply(handling_accented)
clean_text = clean_text.apply(cleaning)
clean_text = clean_text.apply(lemmatization)

In [ ]:
# Vectorization 
# 1. Count vectorizer 
count_vec = CountVectorizer()
bow = count_vec.fit_transform(clean_text).A
pd.DataFrame(bow,columns=count_vec.get_feature_names())

In [ ]:
# 2. Tfidf vectorizer 
tfidf_vec = TfidfVectorizer()
tfidf_val = tfidf_vec.fit_transform(clean_text).A
pd.DataFrame(tfidf_val,columns=tfidf_vec.get_feature_names())

**The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation).
The silhouette ranges from −1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.**

In [ ]:
# [[w1,w2],[]]
sent = clean_text.tolist()
final_text = [sen.split() for sen in sent]  # [[],[]]
final_text

In [ ]:
# word2vec 
word2vec_model = Word2Vec(final_text,min_count=2,window=6)

In [ ]:
# save model 
word2vec_model.save("Word2vec.model")

In [ ]:
# model.wv : vocabulary 

In [ ]:
# doc vect = vect1+ vect2/2
def vectorizer(list_docs,model):
    feature = [] # we will save vector of a document 
    for doc in list_docs : # iterate over all the reviews 
        zero_vector = np.zeros(model.vector_size) #  zero vector for handling key error
        vectors = []
        for word in doc :
            if word in model.wv : # it just checks if word is present or not in models vocabulary
                try : 
                    vectors.append(model.wv[word])
                except KeyError :
                    continue
        if vectors :
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            feature.append(avg_vec)
        else :
            feature.append(zero_vector)
    return feature

In [ ]:
vectorized_docs = vectorizer(final_text,word2vec_model)

In [ ]:
x_emb = np.array(vectorized_docs)
x_emb

In [ ]:
# kmeans 
def build_kmeans(clusters,data):
    kmeans_model = KMeans(n_clusters=clusters)
    y_pred = kmeans_model.fit_predict(data)
    return kmeans_model,y_pred

In [ ]:
# kmeans- count 
bow_kmeans_model,bow_pred = build_kmeans(3,bow)

In [ ]:
# kmeans-tfidf 
tfidf_kmeans_model,tfidf_pred = build_kmeans(3,tfidf_val)

In [ ]:
# kmeans-word2vec
word2vec_kmeans_model,word2vec_pred = build_kmeans(3,x_emb)

In [ ]:
# evaluation with silhouette 
print(f"silhouette score with KMeans- count {silhouette_score(bow,bow_pred)}")
print(f"silhouette score with KMeans- tfidf  {silhouette_score(tfidf_val,tfidf_pred)}")
print(f"silhouette score with KMeans- word2vec {silhouette_score(x_emb,word2vec_pred)}")

In [ ]:
# silhouette visualizeer 
from yellowbrick.cluster import SilhouetteVisualizer

def visualize_silhouette(data,model,title):
    visualizer = SilhouetteVisualizer(model,colors='yellowbrick')
    visualizer.fit(data)
    plt.title(f"Silhouette visualizer with {title}")

In [ ]:
# kmeans count 
visualize_silhouette(bow,bow_kmeans_model,"kmeans-count")   # all our clusters should be on positive. red line is centroid

In [ ]:
# tfidf
visualize_silhouette(tfidf_val,tfidf_kmeans_model,"kmeans-tfidf")

In [ ]:
# word2vec
visualize_silhouette(x_emb,word2vec_kmeans_model,"kmeans-word2vec")

In [ ]:
# target column
word2vec_pred

In [ ]:
# root cause analysis
final_data.translated_review[:3]

In [ ]:
# rake 
from rake_nltk import Rake
rake_extractor = Rake()
rake_extractor.extract_keywords_from_text(final_data.translated_review[0])
keyphrases = rake_extractor.get_ranked_phrases()

In [ ]:
final_data.translated_review[0]

In [ ]:
import yake

In [ ]:
kw_extractor = yake.KeywordExtractor()
keywords = kw_extractor.extract_keywords(final_data.translated_review[0])
keywords

In [ ]:
list_keyphrase = [kw[0] for kw in keywords[:4]]

In [ ]:
list_keyphrase = [sen.split() for sen in list_keyphrase]

In [ ]:
list_keyphrase

In [ ]:
# doc vect = vect1+ vect2/2
def vectorizer(list_docs,model):
    feature = [] # we will save vector of a document 
    for doc in list_docs : # iterate over all the reviews 
        zero_vector = np.zeros(model.vector_size) #  zero vector for handling key error
        vectors = []
        for word in doc :
            if word in model.wv : # it just checks if word is present or not in models vocabulary
                try : 
                    vectors.append(model.wv[word])
                except KeyError :
                    continue
        if vectors :
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            feature.append(avg_vec)
        else :
            feature.append(zero_vector)
    return feature

In [ ]:
keyphrase_vectors = vectorizer(list_keyphrase,word2vec_model)

In [ ]:
vect1 = keyphrase_vectors[0]
vect2 = keyphrase_vectors[1]

In [ ]:
keyphrase_list = [kw[0] for kw in keywords[:2]]

In [ ]:
keyphrase_list

In [ ]:
import numpy as np
from numpy.linalg import norm

# cosine_score = (np.dot(vect1,vect2))/(norm(vect1)*norm(vect2))
dict1 = {}

for index,kw1 in enumerate(keyphrase_list) : # iterate over keyphrases

    for kw2 in keyphrase_list: # iterate over keyphrases
    
        list_keyphrase = list((kw1.split(),kw2.split())) # [[],[]]
        keyphrase_vectors = vectorizer(list_keyphrase,word2vec_model) # vectors extraction of  keyphrases
        vect1 = keyphrase_vectors[0]
        vect2 = keyphrase_vectors[1]
        cosine_score = (np.dot(vect1,vect2))/(norm(vect1)*norm(vect2)) # cosine similarity between each and every keyphrase
        if cosine_score>0.85 : 
            dict1[index] = [kw1,kw2]
            print(dict1)



In [ ]:
dict1

In [ ]:
{'delete' : ['stop deleting', 'deleting the data']} 